In [2]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
library(tidyverse); 
library(lubridate); 
library(scales);
library(magrittr); 
library(dplyr);
})

In [3]:
library(IRdisplay)

display_html(
'<script>  
code_show=true; 
function code_toggle() {
  if (code_show){
    $(\'div.input\').hide();
  } else {
    $(\'div.input\').show();
  }
  code_show = !code_show
}  
$( document ).ready(code_toggle);
</script>
  <form action="javascript:code_toggle()">
    <input type="submit" value="Click here to toggle on/off the raw code.">
 </form>'
)

# Introduction

__May 2, 2022__.   
__May 13, 2022__. 

[Parent QA ticket](https://phabricator.wikimedia.org/T305442) | [QA ticket](https://phabricator.wikimedia.org/T306559)

# Instrumentation note
The test group enrollment events are stored in mediawiki_web_ab_test_enrollment schema.
As of 2022-04-27, user bucketing is based on what page user lands on first, and then persistent for that session. The page_id of the landing page is the salt to decide which test group the session will be in. Session_id is used to identify user session, stored in field web_session_id.

Test population: logged in users and anonymous users
Group split: Sample entire population, 50% to control group, 50% to test group.

__Technical limitation:__  
For the users who land on the wiki page, then click through internal referral, the session will be consistently assigned to one experiment group.

For the users who start a new a tab or open browser again, and land on another wiki page from search, the session will be randomized again. Very likely to be assigned to another experiment group. 

__Check whether we could exclude Chrome 101.0.4951.41__ 

Because we have a bug that links in the TOC of vector 2022 are not clickable for some Chromium based browsers T307271.


In [10]:
query <-
"
SELECT http.request_headers['user-agent']
FROM   event.mediawiki_web_ab_test_enrollment
WHERE year=2022 and month=4
AND meta.domain='he.wikipedia.org' 
AND  http.request_headers['user-agent'] regexp 'Chrome/101.0.4951.41'
LIMIT 5
" 

In [12]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [13]:
df

X_c0
<chr>
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36"
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36"
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36"
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36"
"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36"


# Check experiment group bucketing

QAed on 2022-05-02

In [19]:
query <- "
select wiki, TO_DATE(meta.dt) AS date_time,`group`, count(1) AS pv,
count(distinct web_session_id) AS sessions
FROM event.mediawiki_web_ab_test_enrollment
WHERE year=2022 AND month IN (4,5)
AND wiki IN ('hewiki', 'euwiki')
AND experiment_name='skin-vector-toc-experiment'
GROUP BY wiki, TO_DATE(meta.dt),`group`
ORDER BY wiki, date_time, `group`
LIMIT 100000
"

In [20]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [21]:
df

wiki,date_time,group,pv,sessions
<chr>,<chr>,<chr>,<int>,<int>
euwiki,2022-04-26,control,273,166
euwiki,2022-04-26,treatment,350,179
euwiki,2022-04-27,control,9301,5375
euwiki,2022-04-27,treatment,9022,5326
euwiki,2022-04-28,control,11108,6099
euwiki,2022-04-28,treatment,10456,6089
euwiki,2022-04-29,control,7921,4665
euwiki,2022-04-29,treatment,8335,4818
euwiki,2022-04-30,control,3107,1457


# Check sessions are assigned in both groups

In [8]:
query_check_duplicate <- "
WITH t_duplicate AS
(SELECT  web_session_id, wiki, count(distinct `group` ) AS groups,  min(meta.dt) AS session_dt 
FROM event.mediawiki_web_ab_test_enrollment
WHERE wiki NOT IN ('testwiki','test2wiki')  and year=2022 and month IN (4,5)
GROUP BY  web_session_id, wiki
HAVING groups>1
)
SELECT wiki, count(web_session_id) AS dup_sessions
FROM t_duplicate
GROUP BY wiki
"

In [9]:
df_duplicate <- wmfdata::query_hive(query_check_duplicate)

Don't forget to authenticate with Kerberos using kinit



In [10]:
df_duplicate

wiki,dup_sessions
<chr>,<int>
euwiki,6733
hewiki,170598


In [11]:
query_total_sessions <- "
SELECT  wiki, `group`, 
count(distinct web_session_id) AS sessions
FROM event.mediawiki_web_ab_test_enrollment
WHERE year=2022 AND month IN (4,5)
AND wiki IN ('hewiki', 'euwiki')
AND experiment_name='skin-vector-toc-experiment'
GROUP BY wiki, `group`
ORDER BY wiki, `group`
LIMIT 100000
"

In [12]:
df_total_sessions <- wmfdata::query_hive(query_total_sessions )

Don't forget to authenticate with Kerberos using kinit



In [13]:
df_total_sessions

wiki,group,sessions
<chr>,<chr>,<int>
euwiki,control,25464
euwiki,treatment,25622
hewiki,control,480715
hewiki,treatment,482981


In [14]:
df_sum <- merge(df_total_sessions,df_duplicate,by="wiki")

In [17]:
df_sum <- df_sum %>% 
 mutate(dup_rate =round(dup_sessions/sessions * 100 , 2) )


In [18]:
df_sum

wiki,group,sessions,dup_sessions,dup_rate
<chr>,<chr>,<int>,<int>,<dbl>
euwiki,control,25464,6733,26.44
euwiki,treatment,25622,6733,26.28
hewiki,control,480715,170598,35.49
hewiki,treatment,482981,170598,35.32


## How duplicated sessions distributed by user type ( anaonymous users vs logged-in users)

In [31]:
query_dup_is_anon <- " 
WITH t_dup AS
(SELECT  web_session_id, wiki, count(distinct `group` ) AS groups,  min(meta.dt) AS session_dt 
FROM event.mediawiki_web_ab_test_enrollment
WHERE wiki NOT IN ('testwiki','test2wiki')  and year=2022 and month IN (4,5)
AND experiment_name='skin-vector-toc-experiment'
GROUP BY  web_session_id, wiki
HAVING groups>1
)
SELECT t1.event.token AS web_session_id, t1.event.skinversion, t1.wiki,  t1.event.isAnon, min(TO_DATE(t1.dt)) AS init_date
FROM event.DesktopWebUIActionsTracking AS t1
INNER JOIN t_dup ON t_dup.wiki=t1.wiki AND t_dup.web_session_id=t1.event.token
WHERE t1.wiki NOT IN ('testwiki','test2wiki') 
AND t1.year=2022 and t1.month IN (4,5)
AND t1.event.action='init' 
AND t_dup.session_dt <= t1.meta.dt
GROUP BY t1.event.token , t1.event.skinversion, t1.wiki, t1.event.isAnon
"

In [32]:
df_dup_is_anon <- wmfdata::query_hive(query_dup_is_anon)

Don't forget to authenticate with Kerberos using kinit



In [33]:
df_dup_is_anon_summary <-  df_dup_is_anon  %>%
   group_by(isanon, skinversion) %>%
   summarize(total_isanon_sessions = n(), .groups='drop')

In [34]:
df_dup_is_anon_summary

isanon,skinversion,total_isanon_sessions
<chr>,<int>,<int>
false,1,88
false,2,4353
true,2,109612


# Check whether `mediawiki_web_ab_test_enrollment` can join with click events 

In [39]:
query_click <- "
WITH t_ab_no_dupli AS (
SELECT  web_session_id, wiki, meta.domain AS domain, count(distinct `group` ) AS groups,  min(meta.dt) AS session_dt 
FROM event.mediawiki_web_ab_test_enrollment
WHERE wiki NOT IN ('testwiki','test2wiki')  AND year=2022 AND month IN (4,5) 
AND  experiment_name='skin-vector-toc-experiment'
GROUP BY  web_session_id, wiki, meta.domain
-- exclude session ids are in both control and treatment group
HAVING groups < 2
),
t_ab AS(
SELECT 
 t1.web_session_id,
 t1.wiki,t1.meta.domain AS domain,
 t1.`group` AS test_group,
 min(t1.meta.dt) AS session_dt 
FROM event.mediawiki_web_ab_test_enrollment AS t1
INNER JOIN  t_ab_no_dupli AS t2 ON t1.wiki=t2.wiki 
AND t1.web_session_id=t2.web_session_id 
WHERE t1.wiki NOT IN ('testwiki','test2wiki')  
AND year=2022 AND month IN (4,5) 
AND  experiment_name='skin-vector-toc-experiment'
GROUP BY  t1.web_session_id, t1.wiki,t1.meta.domain, t1.`group`
)
-- clicks from test group
SELECT CONCAT(t1.year, '-', LPAD(t1.month,2,'0'),'-', LPAD(t1.day,2,'0')) AS event_date, 
wiki, event.isanon,  count(1) AS clicks
FROM event.DesktopWebUIActionsTracking AS t1
INNER JOIN t_ab AS t2 
ON  t1.wiki=t2.wiki AND t1.event.token = t2.web_session_id
WHERE t1.wiki IN ('bnwiki', 'fawiki', 'foundationwiki',
'hewiki', 'ptwikinews', 'ptwikiversity', 'srwiki',
'thwiki', 'vecwiki', 'viwiki', 'viwikibooks', 'dewikivoyage',
'euwiki', 'kowiki', 'plwikinews', 'trwiki', 'arywiki',
'frwiki', 'frwikiquote', 'frwiktionary', 'incubatorwiki', 'ptwiki'
) 
AND t1.year=2022 and t1.month IN (4,5)
AND t2.session_dt <= t1.meta.dt
AND event.name = 'ui.toc'
AND event.action='click' AND event.skinversion=2 
GROUP BY t1.year, t1.month,t1.day, t1.wiki, event.isanon
"

In [40]:
df_click <- wmfdata::query_hive(query_click)

Don't forget to authenticate with Kerberos using kinit



In [41]:
df_click

event_date,wiki,isanon,clicks
<chr>,<chr>,<chr>,<int>
2022-04-26,euwiki,true,2
2022-04-26,hewiki,false,11
2022-04-26,hewiki,true,77
2022-04-27,euwiki,false,10
2022-04-27,euwiki,true,103
2022-04-27,hewiki,false,81
2022-04-27,hewiki,true,1213
2022-04-28,euwiki,false,6
2022-04-28,euwiki,true,86


# Check whether `mediawiki_web_ab_test_enrollment` can join with scroll events 

In [50]:
query_scroll <- "
WITH t_ab_no_dupli AS (
SELECT  web_session_id, wiki, meta.domain AS domain, count(distinct `group` ) AS groups,  min(meta.dt) AS session_dt 
FROM event.mediawiki_web_ab_test_enrollment
WHERE wiki NOT IN ('testwiki','test2wiki')  AND year=2022 AND month IN (4,5) 
AND  experiment_name='skin-vector-toc-experiment'
GROUP BY  web_session_id, wiki, meta.domain
-- exclude session ids are in both control and treatment group
HAVING groups < 2
),
t_ab AS(
SELECT 
 t1.web_session_id,
 t1.wiki,t1.meta.domain AS domain,
 t1.`group` AS test_group,
 min(t1.meta.dt) AS session_dt 
FROM event.mediawiki_web_ab_test_enrollment AS t1
INNER JOIN  t_ab_no_dupli AS t2 ON t1.wiki=t2.wiki 
AND t1.web_session_id=t2.web_session_id 
WHERE t1.wiki NOT IN ('testwiki','test2wiki')  
AND year=2022 AND month IN (4,5) 
AND  experiment_name='skin-vector-toc-experiment'
GROUP BY  t1.web_session_id, t1.wiki,t1.meta.domain, t1.`group`
)

--scrolls by session
SELECT  TO_DATE(t3.meta.dt) AS event_date,t_ab.wiki,   t_ab.test_group, t3.web_session_id, COUNT(1) AS scrolls 
FROM t_ab
LEFT JOIN event.mediawiki_web_ui_scroll AS t3 
ON t_ab.domain=t3.meta.domain AND t3.web_session_id = t_ab.web_session_id
AND year=2022 AND month IN (4,5) 
WHERE t_ab.session_dt <= t3.meta.dt
AND t3.action='scroll-to-toc'
GROUP BY  TO_DATE(t3.meta.dt) , t_ab.wiki,   t_ab.test_group, t3.web_session_id

"




In [43]:
df_scroll <- wmfdata::query_hive(query_scroll)

Don't forget to authenticate with Kerberos using kinit



In [47]:
df_scroll_summary <- df_scroll %>%
  group_by(wiki, test_group) %>% 
  summarize(sum_scroll=sum(scrolls), sum_session=n())

`summarise()` has grouped output by 'wiki'. You can override using the `.groups` argument.



In [48]:
df_scroll_summary

wiki,test_group,sum_scroll,sum_session
<chr>,<chr>,<int>,<int>
euwiki,control,2854,904
euwiki,treatment,1446,769
hewiki,control,32393,13810
hewiki,treatment,19618,12245


__NOTE__  

Next step:
Add total number of sessions (pageviews), see how the trend is difference with the number of unique sessions. 

QA on May 13, 2022, for the fix in scroll schema, enabled on 5/12. 
[fix ticket](https://phabricator.wikimedia.org/T307952)

In [64]:
query_scroll <- "
WITH t_ab_no_dupli AS (
SELECT  web_session_id, wiki, meta.domain AS domain, count(distinct `group` ) AS groups,  min(meta.dt) AS session_dt 
FROM event.mediawiki_web_ab_test_enrollment
WHERE wiki NOT IN ('testwiki','test2wiki')  AND year=2022 AND month IN (4,5)
AND  experiment_name='skin-vector-toc-experiment'
GROUP BY  web_session_id, wiki, meta.domain
-- exclude session ids are in both control and treatment group
HAVING groups < 2
),
t_ab AS(
SELECT 
 t1.web_session_id,
 t1.wiki,t1.meta.domain AS domain,
 t1.`group` AS test_group,
 min(t1.meta.dt) AS session_dt 
FROM event.mediawiki_web_ab_test_enrollment AS t1
INNER JOIN  t_ab_no_dupli AS t2 ON t1.wiki=t2.wiki 
AND t1.web_session_id=t2.web_session_id 
WHERE t1.wiki NOT IN ('testwiki','test2wiki')  
AND year=2022 AND month=5 AND day IN (12,13) 
AND  experiment_name='skin-vector-toc-experiment'
GROUP BY  t1.web_session_id, t1.wiki,t1.meta.domain, t1.`group`
)

--scrolls by session
SELECT  TO_DATE(t3.meta.dt) AS event_date,t_ab.wiki,   t_ab.test_group, t3.web_session_id, COUNT(1) AS scrolls 
FROM t_ab
LEFT JOIN event.mediawiki_web_ui_scroll AS t3 
ON t_ab.domain=t3.meta.domain AND t3.web_session_id = t_ab.web_session_id
AND year=2022 AND month=5 AND day IN (12,13) 
WHERE t_ab.session_dt <= t3.meta.dt
AND t3.action='scroll-to-toc'
GROUP BY  TO_DATE(t3.meta.dt) , t_ab.wiki,   t_ab.test_group, t3.web_session_id

"



In [65]:
df_scroll <- wmfdata::query_hive(query_scroll)

Don't forget to authenticate with Kerberos using kinit



In [66]:
df_scroll %>%
  group_by(wiki, test_group) %>% 
  summarize(sum_scroll=sum(scrolls), sum_session=n())

`summarise()` has grouped output by 'wiki'. You can override using the `.groups` argument.



wiki,test_group,sum_scroll,sum_session
<chr>,<chr>,<int>,<int>
euwiki,control,608,255
euwiki,treatment,645,288
hewiki,control,4359,2711
hewiki,treatment,4359,2729


# Check whether `mediawiki_web_ab_test_enrollment` can join with reading depth events 

In [53]:
query_reading_length <- "
WITH t_ab_no_dupli AS (
SELECT  web_session_id, wiki, meta.domain AS domain, count(distinct `group` ) AS groups,  min(meta.dt) AS session_dt 
FROM event.mediawiki_web_ab_test_enrollment
WHERE wiki NOT IN ('testwiki','test2wiki')  AND year=2022 AND month IN (4,5) 
AND  experiment_name='skin-vector-toc-experiment'
GROUP BY  web_session_id, wiki, meta.domain
-- exclude session ids are in both control and treatment group
HAVING groups < 2
),
t_ab AS(
SELECT 
 t1.web_session_id,
 t1.wiki,t1.meta.domain AS domain,
 t1.`group` AS test_group,
 min(t1.meta.dt) AS session_dt 
FROM event.mediawiki_web_ab_test_enrollment AS t1
INNER JOIN  t_ab_no_dupli AS t2 ON t1.wiki=t2.wiki 
AND t1.web_session_id=t2.web_session_id 
WHERE t1.wiki NOT IN ('testwiki','test2wiki')  
AND year=2022 AND month IN (4,5) 
AND  experiment_name='skin-vector-toc-experiment'
GROUP BY  t1.web_session_id, t1.wiki,t1.meta.domain, t1.`group`
)

--reading time by session
SELECT  TO_DATE(t3.meta.dt) AS event_date,t_ab.wiki,   t_ab.test_group, t3.session_token, is_anon,
sum(total_length) AS sum_reading_length, sum(page_length) AS sum_page_length
FROM event.mediawiki_reading_depth AS t3 
INNER JOIN  t_ab
ON t_ab.domain=t3.meta.domain AND t3.session_token = t_ab.web_session_id
WHERE year=2022 AND month IN (4,5) 
AND t_ab.session_dt <= t3.meta.dt
AND t3.meta.domain IN (
'he.wikipedia.org',
'eu.wikipedia.org'
)
GROUP BY  TO_DATE(t3.meta.dt) , t_ab.wiki,   t_ab.test_group, t3.session_token, is_anon

"

In [54]:
query_reading <- wmfdata::query_hive(query_reading_length) 

Don't forget to authenticate with Kerberos using kinit



QA on 2022-05-12

## check whether we can identify user_agent_map['device_family']='Spider' 

In [61]:
query <-
"
SELECT meta.domain, `group`, count(distinct web_session_id) AS sessions, count(1) AS events
FROM   event.mediawiki_web_ab_test_enrollment
WHERE year=2022 and month=4
AND user_agent_map['device_family']='Spider' 
GROUP BY meta.domain, `group`
LIMIT 5
" 

In [62]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [63]:
df

domain,group,sessions,events
<chr>,<chr>,<int>,<int>
he.wikipedia.org,control,15,28
he.wikipedia.org,treatment,16,35
